In [4]:
# This notebook downloads archived quarterly SEC Form 4 ZIP files from sec.gov.
# The data is stored locally for further processing and analysis.

import requests
import os

#Get the current directory of the notebook
notebook_dir = os.getcwd()
#Explicity make sure to make this the working dirctory for consistency
os.chdir(notebook_dir)
print(f'Current working directory: {os.getcwd()}')

#Create a function to download SEC Form 4 ZIP files
def download_sec_zips(save_dir="sec_zips"): # Saves to local session directory. Modify to change save directory.
    os.makedirs(save_dir, exist_ok=True)
    base_url = "https://www.sec.gov/files/structureddata/data/insider-transactions-data-sets" # Location of archived Form 4s.
    headers = {"User-Agent": "kirtland@umich.edu"}  # SEC blocks anonymous requests; requires a valid email. Change email as needed.

    failed = []

    for year in range(2006, 2026):
        for q in range(1, 5):
            if year == 2025 and q > 1:
                break  # Only through 2025 Q1, which is most recent.

            filename = f"{year}q{q}_form345.zip"
            url = f"{base_url}/{filename}"
            local_path = os.path.join(save_dir, filename)

            try:
                r = requests.get(url, headers=headers, timeout=30)
                if r.status_code == 200:  # 200 means download successful
                    with open(local_path, "wb") as f:
                        f.write(r.content)
                    print(f"Downloaded: {filename}")
                else:
                    print(f"Failed: {filename} (status {r.status_code})")
                    failed.append(filename)
            except Exception as e:
                print(f"Error downloading {filename}: {e}")
                failed.append(filename)

    if failed:
        print("\n Some files failed to download:")
        for f in failed:
            print(" -", f)
    else:
        print("\n All zip files downloaded successfully.")


Current working directory: c:\Users\tmacp\OneDrive\Documents\VSCode\River Rose Financial\Portfolio\Insider Trading


In [5]:
download_sec_zips()

Downloaded: 2006q1_form345.zip
Downloaded: 2006q2_form345.zip
Downloaded: 2006q3_form345.zip
Downloaded: 2006q4_form345.zip
Downloaded: 2007q1_form345.zip
Downloaded: 2007q2_form345.zip
Downloaded: 2007q3_form345.zip
Downloaded: 2007q4_form345.zip
Downloaded: 2008q1_form345.zip
Downloaded: 2008q2_form345.zip
Downloaded: 2008q3_form345.zip
Downloaded: 2008q4_form345.zip
Downloaded: 2009q1_form345.zip
Downloaded: 2009q2_form345.zip
Downloaded: 2009q3_form345.zip
Downloaded: 2009q4_form345.zip
Downloaded: 2010q1_form345.zip
Downloaded: 2010q2_form345.zip
Downloaded: 2010q3_form345.zip
Downloaded: 2010q4_form345.zip
Downloaded: 2011q1_form345.zip
Downloaded: 2011q2_form345.zip
Downloaded: 2011q3_form345.zip
Downloaded: 2011q4_form345.zip
Downloaded: 2012q1_form345.zip
Downloaded: 2012q2_form345.zip
Downloaded: 2012q3_form345.zip
Downloaded: 2012q4_form345.zip
Downloaded: 2013q1_form345.zip
Downloaded: 2013q2_form345.zip
Downloaded: 2013q3_form345.zip
Downloaded: 2013q4_form345.zip
Download